In [41]:
import numpy as np
import math
np.seterr(divide='ignore', invalid='ignore')
from re import findall
from hazm import Stemmer, word_tokenize
from sklearn.feature_selection import chi2, SelectKBest
from glob import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

"""Countvectorzer initialize"""
vect = CountVectorizer()
is_file = True

"""stemmer from hazm"""
stemmer = Stemmer()
stopwords = set(open('stop-words.txt', encoding='utf8').read().splitlines())
hamtrains = glob('dataset/hamtraining/hamtraining*.txt')
spamtrains = glob('dataset/spamtraining/spamtraining*.txt')

hamtests = glob('dataset/hamtesting/hamtesting*.txt')
spamtests = glob('dataset/spamtesting/spamtesting*.txt')

ham_spam_test = hamtests + spamtests
ham_spam_train = hamtrains + spamtrains

y_train = [False for i in range(300)]
y_spam_train = [True for i in range(300)]
y_train.extend(y_spam_train)


y_test = [False for i in range(200)]
y_spam_test = [True for i in range(200)]
y_test.extend(y_spam_test)

In [42]:
import sys


In [43]:
import sys

In [44]:
!{sys.executable} -m pip install numpy


In [45]:
!{sys.executable} -m pip install hazm

### Preprocess

In [46]:
class const:
    farsi = ('ب', 'س', 'ش', 'ل', 'ت', 'ن', 'م', 'گ', 'ظ', 'ط', 'ز',
             'ر', 'ژ', 'ذ', 'د', 'پ', 'چ', 'ج', 'ح', 'ع', 
             'خ', 'غ', 'ف', 'ق', 'ث', 'ص', 'ض','\u0020', '\u060c','؟', '!', '?', '.', ':','\n','_')

    alef = ('ا', 'آ', 'ء', 'أ', 'إ')
    vav = ('و', 'ؤ')
    heh = ('ه', 'ة', 'ە')
    yah = ('ی', 'ي', 'ئ', 'ى')
    kaf = ('ک', 'ك')
    punc = ('_', '-')

def persian_char(char):
    if char in const.farsi:
        return char
    if char in const.alef:
        return const.alef[0]
    if char in const.vav:
        return const.vav[0]
    if char in const.heh:
        return const.heh[0]
    if char in const.yah:
        return const.yah[0]
    if char in const.kaf:
        return const.kaf[0]
    if char in const.punc:
      return ' '
    return ''

def pre_process(path):
    if is_file == False:
        text = path
    else:
        text = open(path, encoding='utf8').read()

#     urls = len(findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', text))
    
    map_test = map(persian_char, text)
    sentence = ''.join(map_test)
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if w not in stopwords] #+ ['url' for i in range(urls)]
    filtered_sentence = ' '.join(filtered_sentence)
    return filtered_sentence

In [47]:
def feature(data):
    if is_file == False:
        feature_list = [pre_process(data)]
    else:    
        map_loop = map(pre_process, data)
        feature_list = list(map_loop)
    return feature_list

In [48]:
def vectorize(feature_list):
    X_dtm = vect.fit_transform(feature_list)
    X_dtm = X_dtm.toarray()
    return X_dtm

In [49]:
def feature_selection(k,X_dtm):
    chi2_features = SelectKBest(chi2, k=k)
    X_kbest_features = chi2_features.fit_transform(X_dtm, y_train)
    return X_kbest_features, chi2_features

In [50]:
def transform(data,chi2_features):
    tokens = feature(data)
    x0 = vect.transform(tokens).toarray()
    cx = chi2_features.transform(x0)
    return cx

In [51]:
X_dtm = vectorize(feature(ham_spam_train))
x_train, chi2_features = feature_selection(700,X_dtm)

### Just for Intuition

In [52]:
# gnb = GaussianNB()
mnb = MultinomialNB()
mnb.fit(x_train,y_train)

MultinomialNB()

In [53]:
"""Just for test and compare"""
x_test = transform(ham_spam_test, chi2_features)
predict_val = mnb.predict(x_test)

In [54]:
"""Show metrics and score!"""
print(classification_report(y_test, predict_val))

              precision    recall  f1-score   support

       False       0.93      0.97      0.95       200
        True       0.97      0.93      0.95       200

    accuracy                           0.95       400
   macro avg       0.95      0.95      0.95       400
weighted avg       0.95      0.95      0.95       400



### Naive bayes model

<h2 dir="rtl"> ویدیو بسیار مفید برای ساخت مدل<h2>
<h2><a href="https://www.youtube.com/watch?v=O2L2Uv9pdDA">Naive Bayes, Clearly Explained!!!</a><h2>

<h3 dir="rtl">خرجی تابع
transform 
وکتورایز شده هر یک از سمپل‌های موجود در دیتاست تست است.
به طوری که تعداد اعضای 
test_data
برابر با 400 نمونه متنی است که برای تست کردن مدل مورد استفاده قرار میگیرد.
هر یک از سمپل‌ها دارای وکتوری به طول 700 میباشد، که تعداد فیچرهای(کلمات) برتر انتخاب شده توسط تابع SelectKBest 
اند.

در این وکتور 700 بعدی مقدار هر یک از اندیس‌ها نشان دهنده تکرار آن کلمه در آن سمپل خاص هست.

راهنمایی ساخت جدول احتمالات:
شما با کمک تکرار کلمات در هر سمپل و اینکه هر کلمه در کل نمونه‌های 
spam
یا
ham
چند بار تکرار شده است میتوانید احتمال رخدادن هر کلمه را در هر دسته بدست آورید
که در نهایت با ضرب احتمالات بدست آمده تعیین میکنید که ایمیل 
spam
است یا نه.

به این شکل که بعد از ضرب احتمالات در هم دیگر؛ اگر مقدار احتمال دسته 
spam 
بیشتر از دسته 
ham(nonspam)
بود،
آنگاه آن ایمیل spam 
است.
<h3>

In [55]:
test_data = transform(ham_spam_test, chi2_features)

In [56]:
test_data[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

<h3 dir="rtl"> 
شما باید با کمک این فیچر‌ها مدل 
naive bayes 
را برای تشخیص ایمیل‌های 
spam, nonspam
آماده سازی کنید.
خروجی کار شما باید لیبل‌های باشد که تعیین میکند،
ایمیل 
spam هست
یا نه.

<h3>

<h1>Please implement your naive bayse model in here.<h1>

In [57]:
### START CODE HERE ### 
#Build your naive bayes classifier!
def bayes(test_data):

    spam_vec=np.array(test_data)
    ham_vec=np.array(test_data)         #now we have to divide each word into the total
                                    #  number of   spam words and do the same for ham words   
    for i in range(0,len(test_data)):
        spam_vec[i]=test_data[i]/x_train  
        ham_vec[i]=test_data[i]/y_train

    multi_spam = 1                  #it is better to use log for every answer 
                                    # to prevent overflow (because of multiplying each probability)
                                    # and use exp for final answer
    for x in range(0,spam_vec):
            multi= math.log( multi) * math.log( x)

    multi_ham=1
    for x in range(0,ham_vec):
            multi_ham= math.log(multi_ham) *math.log( x)

    #the probability of being spam:
    spam=math.exp((300/600)*multi_spam)
    ham=math.exp((300/600)*multi_ham)

    if spam > ham:
        print("this a spam")
    else:
        print("this a ham")

### END CODE HERE ###

In [58]:
### START CODE HERE ### 
bayes(test_data)

### END CODE HERE ###

### Result

In [59]:
"""Show metrics and score!"""
print(classification_report(y_test, y_predict))

NameError: name 'y_predict' is not defined